<div >
<img src = "../banner.jpg" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento/BDML_202501/blob/main/Modulo08/01_CuadernoModulo08_PCA_Text_as_Data.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Análisis de Componentes Principales (PCA)

Para ilustrar el uso de PCA, usamos el conjunto de datos del paper de Duflo, Dupas y Kremer (2011): ["Peer Effects, Teacher Incentives, and the Impact of Tracking: Evidence from a Randomized Evaluation in Kenya"](https://www.aeaweb.org/articles?id=10.1257/aer.101.5.1739)

**Abstract**

To the extent that students benefit from high-achieving peers, tracking will help strong students and hurt weak ones. However, all students may benefit if tracking allows teachers to better tailor their instruction level. Lower-achieving pupils are particularly likely to benefit from tracking when teachers have incentives to teach to the top of the distribution. We propose a simple model nesting these effects and test its implications in a randomized tracking experiment conducted with 121 primary schools in Kenya. While the direct effect of high-achieving peers is positive, tracking benefited lower-achieving pupils indirectly by allowing teachers to teach to their level. (JEL I21, J45, O15) 

In [ ]:
require("pacman")
p_load("tidyverse","rio")

In [ ]:
dta<-import("https://raw.githubusercontent.com/ignaciomsarmiento/datasets/refs/heads/main/duflo_data.csv")

In [ ]:
summary(dta)

En este paper se reportan puntajes en siete secciones de un examen (cuatro de lenguaje y tres de matemáticas).

Variables:

- pupilid        : ID of student
- wordscore      : Endline Score on Word Recognition section (max: 24)
- sentscore      : Endline Score on Sentence Recognition section (max: 40 )
- letterscore    : Endline Score on Letter Recognition section (max: 70)
- spellscore     : Endline Score on Spelling section (max: 10)
- additions_score: Endline Score on Additions section
- substractionscore: Endline Score on Substractions section
- multiplicationscore: Endline Score on Multiplications section


In [ ]:
cor(dta[,2:8])

In [ ]:
# get rid of pupilid
dta<-dta[,2:8]

# PCA
res_pca <- prcomp(dta, scale=TRUE)
res_pca

In [ ]:
p_load("factoextra")
eig_val <- get_eigenvalue(res_pca)
eig_val

In [ ]:
sum(eig_val$eigenvalue)

Los autovalores suman 7 (una por cada variable). Por ejemplo:

* El **primer autovalor** es aproximadamente 4.0, lo que indica que la primera componente explica alrededor del **57% de la varianza** total.
* El **segundo autovalor** es cercano a 1.0, y explica un **15% adicional**.

En total, las **dos primeras componentes** explican el **72% de la varianza** en los siete puntajes.

In [ ]:
fviz_eig(res_pca, addlabels = TRUE, ylim = c(0, 70))

También podemos visualizar la varianza acumulada:

In [ ]:
eig_val$dim<-seq(1:7)
ggplot(data=eig_val,aes(x=dim,y=cumulative.variance.percent)) +
    geom_point()+
    geom_line() +
    theme_bw()


## Interpretación de las componentes principales


In [ ]:
res_pca


En la **Tabla de vectores propios**  vemos los **pesos (cargas)** de cada variable sobre cada componente:

### PC1

Para la **primera componente**, todos los pesos son **positivos y similares**, lo que sugiere que esta componente es muy parecida a un **promedio simple** de los siete puntajes. Es decir, la primera componente captura la mayoria de la información contenida en los 7 puntajes y resume el rendimiento general del estudiante.

Que todos los pesos tengan un mismo signo (no importa cual es) nos dice que hay una dimensión común de rendimiento que domina la variabilidad en los datos. En lugar de tratar de interpretar siete puntajes distintos, podemos resumirlos de forma bastante precisa usando una sola variable: la primera componente principal.


### PC2

Para la **segunda componente**, los pesos tienen un patron diferente. Los **cuatro puntajes de lenguaje** tienen pesos negativos y los **tres de matemáticas**, positivos. 

Esta componente está capturando una contraposición o contraste entre dos grupos de variables.

Esto indica que esta componente representa algo como la **diferencia relativa entre habilidades matemáticas y verbales**.



Entonces 


* El **57% de la variación** en los puntajes puede explicarse por una sola variable resumen: el promedio general.
* El **72% de la variación** se puede explicar si consideramos además por separado los puntajes en matemáticas y lenguaje.

En otras palabras, con solo dos componentes principales capturamos casi tres cuartas partes de la información contenida en los siete puntajes individuales.

### Biplot

El biplot es una herramienta visual muy útil en el análisis de componentes principales, ya que permite observar en un mismo gráfico:

 * A los individuos (en este caso, los estudiantes) proyectados en el espacio de los primeros dos componentes principales.

 * A las variables originales (los siete puntajes del test), representadas como vectores.

In [ ]:
set.seed(123)  # Fijar semilla para reproducibilidad


# Seleccionar algunos individuos aleatoriamente (faster)
subset_ids <- sample(1:dim(dta)[1], 30)

fviz_pca_biplot(res_pca, 
                repel = TRUE,# Avoid text overlapping
                col.var = "#2E9FDF", # Variables color
                col.ind = "#696969", #"white" #  Individuals color
                select.ind = list(name = subset_ids)
                )

#### ¿Cómo se interpreta?

**1. Posición de los individuos (puntos grises):**
Cada punto representa a un estudiante. Estos son los "scores" o "puntajes principales" de los componentes. Son los valores proyectados sobre PC1 y PC2. Entonces su ubicación refleja su posición en las primeras dos componentes principales:

* Estudiantes que están cerca del centro tienen un rendimiento más promedio.
* Estudiantes alejados del centro tienen rendimientos más extremos (positivos o negativos).
* La dirección de su posición indica si destacan más en habilidades verbales o matemáticas.

Estudiantes que aparecen cerca del centro del biplot (el punto 0,0) no se desvían mucho del promedio del grupo, ni en el rendimiento general (captado por la primera componente), ni en la diferencia entre áreas (captada por la segunda componente). Es decir, tienen un perfil académico bastante "típico" o balanceado.

In [ ]:
dta[2567,]

In [ ]:
summary(dta)

In [ ]:
dta[953,]



**2. Dirección y longitud de las flechas (variables):**
Cada flecha representa una de las variables originales (los puntajes en secciones del test).

* **La dirección de la flecha** indica hacia dónde crece el valor de esa variable.
* **La longitud de la flecha** indica el peso de la variable en el componente

**3. Ángulo entre flechas:**

* Si dos flechas apuntan en la **misma dirección**, las variables están **positivamente correlacionadas**.
* Si forman un **ángulo de 90°**, las variables son **no correlacionadas**.
* Si apuntan en **direcciones opuestas**, están **negativamente correlacionadas**.

En nuestro caso

* Las flechas correspondientes a los **puntajes de lenguaje**  estén agrupadas y apunten en una dirección similar.
* Lo mismo ocurre con las de **matemáticas**, pero en dirección opuesta a las de lenguaje en la segunda componente.
* Esto refuerza la interpretación de la segunda componente como una especie de contraste entre habilidades matemáticas y verbales.



##  Votos en el congreso de Estados Unidos

Este es un ejemplo un poco mas complejo de como votan los miembros del Congreso de Estados Unidos. 

Tenemso información de los votos del 111vo Congreso que cubre los años 2009 y 2010, que son los dos primeros años de la presidencia de Obama.

Hay 445 miembros que votan en la casa de representantes (US House of Representatives) y se registraron 1647 votos con -1 por el negativo, +1 por positivo, y 0 por la abstencion o la ausencia

In [ ]:
votes <- read.csv("https://github.com/ignaciomsarmiento/datasets/raw/main/rollcall-votes.csv")
head(votes)

In [ ]:
dim(votes)

In [ ]:
legis <- read.csv("https://github.com/ignaciomsarmiento/datasets/raw/main/rollcall-members.csv")
head(legis)

En este contexto podriamos pensar para una estructura factorial subjacente de baja dimensión. Aunque cada uno de los votos se refiere a temas diferentes, los representantes suelen están alineados según ejes partidistas e ideológicos (por ejemplo, republicano versus demócrata o liberal versus conservador). Si creemos que todos los votos son partidistas, entonces el voto por el miembro i en el tema j podría predecirse como

$$
x_{i} = h f_i
$$

donde f es el posicionamiento ideologico latente y h como este posicionamiento afecta los votos. Este tipo de situaciones es ideal para PCA

In [ ]:
#Estimamos PCA
pcavote <- prcomp(votes, scale=TRUE)

pcavote$rotation

In [ ]:
fviz_eig(pcavote, addlabels = TRUE, ylim = c(0, 40))

In [ ]:
fviz_pca_biplot(pcavote)

In [ ]:
fviz_pca_biplot(pcavote, 
                col.ind = legis$party,
                palette = c("blue", "green", "red"),
                invisible ="var",
                repel=FALSE,
                labelsize = 1
                )

In [ ]:
votepc <- predict(pcavote) #
votepc[order(votepc[,1])[1:5],1]

In [ ]:
votepc[order(-votepc[,1])[1:5],1]

In [ ]:
votepc[order(votepc[,2])[1:5],2]

In [ ]:
votepc[order(-votepc[,2])[1:5],2]

In [ ]:
loadings <- pcavote$rotation[,1:2]

In [ ]:
hist(loadings[,1], main="", xlab="1st Principle Component Vote-Loadings",
     col=8, border=grey(.9))
abline(v=loadings[884,1], col=2)
text(x=loadings[884,1], y=550, "Afford. Health (amdt.)", xpd=TRUE, col=2, font=3)
abline(v=loadings[25,1], col=4)
text(x=loadings[25,1], y=550, "TARP", xpd=TRUE, col=4, font=3)

In [ ]:
loadings[order(abs(loadings[,2]), decreasing=TRUE)[1:5],2]

In [ ]:
sort(rowSums(votes==0), decreasing=TRUE)[1:5]

# Regresión por Componentes Principales (PCR)

Ahora que ya sabemos cómo ajustar modelos de factores, ¿para qué sirven?

- En algunos contextos, como el ejemplo de ciencias políticas, los factores tienen un significado claro y ayudan a entender sistemas complejos.
- Más comúnmente, sin embargo, los factores pueden no tener una interpretación clara.
- Aun así, pueden ser útiles como insumos en un sistema de regresión.
- De hecho, esta es una de las principales aplicaciones prácticas del PCA: ser la primera etapa en la Regresión por Componentes Principales (PCR).



La idea detrás de la PCR es simple:

- En lugar de ajustar un modelo \( y \sim X \), 
- Se usan un conjunto reducido de componentes principales (PCs) como regresores.

Esto es útil por varias razones:

- El PCA reduce la dimensionalidad del problema, lo cual suele ser beneficioso.
- Los componentes principales son independientes, por lo tanto, no hay multicolinealidad y la regresión final es más fácil de ajustar.

## Algoritmo paso a paso de la PCR

El procedimiento es sencillo:

1. Aplicar PCA a las variables predictoras \( X \)
2. Extraer los primeros K componentes principales
3. Ajustar una regresión lineal de y sobre los componentes

### Limitaciones de la PCR

- La PCR se basa en los componentes principales de \( X \), que capturan las mayores fuentes de variación en los datos.
- Si la variable de interés \( y \) está relacionada con esas fuentes dominantes de variación, la PCR funcionará bien.
- Pero si \( y \) depende de una pequeña cantidad de variables específicas (una "aguja en un pajar"), la PCR podría no funcionar bien.
- En la práctica, es difícil saber de antemano qué escenario se tiene. Por eso, conviene comparar PCR con otras alternativas como LASSO directamente sobre los \( X \).

## ¿Cuántos componentes principales usar?

Cuando usamos PCA solo para reducir la dimensión, teníamos algunas reglas para elegir el número de componentes.

¿Deberíamos aplicar lo mismo aquí?

En la PCR el enfoque es un poco diferente:

- Se calculan \( min(n-1, p) \) componentes.
- Luego se hace validación cruzada (por ejemplo K-fold) agregando un PC a la vez.
- Se elige el modelo que minimiza el error cuadrático medio fuera de muestra (MSE).

Como los PCs están ordenados por varianza explicada y son independientes, este procedimiento suele funcionar mejor que seleccionar subconjuntos directamente sobre las variables originales.

### Estrategias híbridas con LASSO

- Una alternativa práctica: ajustar una regresión LASSO sobre todos los componentes principales.
- Este enfoque facilita la inclusión de información adicional.

Por ejemplo:

- Incluir tanto los PCs como las variables \( X \) originales en la matriz del modelo LASSO.
- Esto permite que el modelo aproveche tanto la estructura latente (los factores) como señales específicas de variables individuales relacionadas con \( y \).
- Esta estrategia híbrida soluciona la principal desventaja de la PCR: que sólo captura las principales fuentes de variación de \( X \).


## Ejemplo prediccion de ratings

Para ilustrar vamos a usar datos de television que incluyen respuestas a encuestas para grupos focales sobre programas piloto de televisión (primeros episodios de una nueva serie), así como los resultados del primer año de calificaciones (cuántas personas terminaron viendo el programa). 

La esperanza es que podamos a predecir el interés de los espectadores a partir de encuestas piloto, ayudando así a los estudios a tomar mejores decisiones de programación. 


## Datos de programas

In [ ]:
shows <- read.csv("https://github.com/ignaciomsarmiento/datasets/raw/main/nbc_showdetails.csv", row.names=1)
shows$Genre <- factor(shows$Genre)
head(shows)

Tenemos un par de variables de resultado interesantes. 
- Los puntos de rating bruto (GRP) proporcionan un recuento estimado de la audiencia total. 
- Compromiso proyectado (PE) como una medida más sutil de la atención de la audiencia.

Después de ver un programa, se contacta a los espectadores y se les pregunta sobre el orden y los detalles de los eventos del programa. Esto mide su compromiso con el programa (y, quizás lo más importante, con los anuncios mostrados). La PE se informa en una escala de 0 a 100, donde 100 significa que estaban completamente comprometidos y 0 significa que no prestaron atención en absoluto.


In [ ]:
plot(GRP ~ PE, data=shows, bg=c(4,2,3)[shows$Genre], pch=21, log="y")
legend("bottomright", legend=levels(shows$Genre), fill=c(4,2,3), bty="n")

La figura mustra que en general una mayor interacción suele corresponder a mayores índices de audiencia, pero que las comedias pueden tener una alta interacción con índices de rating más bajos. Los realities tienden a tener menor interacción y audiencia (pero son económicos de producir).

## Datos de la encuesta

Los datos de la encuesta incluyen 6241 vistas y 20 preguntas para 40 programas. Hay dos tipos de preguntas en la encuesta. Ambos  preguntan hasta qué punto estás de acuerdo con una afirmación. 

- Para la primer grupo de preguntas (Q1) esta afirmación toma la forma de "Este programa me hace sentir...".
- Para el segundo grupo de preguntas (Q2), la afirmación es "Encuentro que este programa me hace sentir...". 



In [ ]:
survey <- read.csv("https://github.com/ignaciomsarmiento/datasets/raw/main/nbc_pilotsurvey.csv", as.is=TRUE) 
survey$Show <- factor(survey$Show, levels=rownames(shows))
head(survey)

In [ ]:
dim(survey)

Pareciera que tenemos muchos datos 6231 personas que vieron el pilot, per en realidad solo hay 40  programas con 20 preguntas y hay dos variables potenciales a predecir: GRP y PE.

Para relacionar los resultados de las encuestas al performance del programa, primero tenemos que calcuar el promedio para las respuestas de la encuesta por show:

In [ ]:
Xpilot <- aggregate(survey[,-(1:2)],  ## -(1:2) to remove the variables 'show' and 'viewer' completely
                by=list(Show=survey$Show), mean)


rownames(Xpilot) <- Xpilot[,1]
Xpilot <- Xpilot[,-1]
head(Xpilot)

Esto nos deja con una matris X con las siguientes dimensions

In [ ]:
dim(Xpilot)

Podemos hacer PCA

In [ ]:
PCApilot <- prcomp(Xpilot, scale=TRUE)

fviz_eig(PCApilot, addlabels = TRUE, ylim = c(0, 45))

Veamos los 2 primerosloadings o rotaciones:

In [ ]:
round(PCApilot$rotation[,1:3],2)

El PC1 parece que es un factor que captura "cuan mucho le disgusta el programa" (el negativo sería cuan mucho me gusta). Un programa obtiene una puntuación baja en el primer componente principal (PC1) si hizo que los espectadores se sintieran emocionados y comprometidos, y si el contenido era original, entretenido y con suspenso. Un programa obtiene una puntuación alta en PC1 si las personas lo encontraron molesto y aburrido.

El segundo componente principal (PC2) es menos fácil de interpretar: obtienes una puntuación alta en PC2 si encuentras el programa aburrido, confuso y predecible, pero también si lo encuentras gracioso.

Veamos si un Biplot nos puede ayudar un poco mas: 

In [ ]:
fviz_pca_biplot(PCApilot, 
                col.ind = shows$Genre,
                palette = c("blue", "green", "red"),
                ylim=c(-3,3),
                xlim=c(-6,6), # hides "monarch cove",living with ed", and "next" but these are all tiny 
                invisible ="var",
                repel=TRUE,
                labelsize = 2
                )


Los programas de "reality" obtienen loadings altos tanto en PC1 como en PC2—son poco agradables y pueden ser molestos pero graciosos—mientras que los dramas con guion obtienen puntuaciones bajas en ambos.


## Lasso Regressions 

Veamos ahora si podemos predecir engagement (PE)

In [ ]:
p_load("gamlr")

In [ ]:
zpilot <- predict(PCApilot)

PE <- shows$PE
zdf <- as.data.frame(zpilot)

summary(PEglm <- glm(PE ~ ., data=zdf[,1:2]))

### Solo X

In [ ]:
cvlasso <- cv.gamlr(x=as.matrix(Xpilot), y=PE, nfold=10)
coef(cvlasso) 

### Solo PCs

In [ ]:
cvlassoPCR <- cv.gamlr(x=zpilot, y=PE, nfold=10) 
coef(cvlassoPCR) 

### Ambos

In [ ]:
cvlassoboth <- cv.gamlr(x=as.matrix(cbind(Xpilot,zpilot)), y=PE, nfold=10)
coef(cvlassoboth)

In [ ]:
par(mfrow=c(1,3), mai=c(.2,.2,.5,.1), omi=c(.5,.5,0,0))
plot(cvlasso, main="Lasso on X", ylim=c(50,200), ylab="", xlab="", df=FALSE, bty="n")
plot(cvlassoPCR, main="Lasso on PCR", ylim=c(50,200), ylab="", xlab="", df=FALSE, bty="n")
plot(cvlassoboth, main="Lasso on X and PCR", ylim=c(50,200), ylab="", xlab="", df=FALSE, bty="n")
mtext(side=2, "mean squared error", outer=TRUE, line=2)
mtext(side=1, "log lamba", outer=TRUE, line=2)

# Texto como Datos: Ejemplos

## Regresión con Texto: Gentzkow and Shapiro

Vamos a usar los datos del estudio de Gentzkow y Shapiro [2010], que analiza cómo los periódicos ajustan su contenido según la orientación política de sus lectores.


<div >
<img src = "figures/gentzgow_shapiro.png" />
</div>

Cargamos los paquetes

In [ ]:
require("pacman")
p_load("tidyverse","textir","wordcloud")

El conjunto de datos resume el primer año del 109º Congreso de los Estados Unidos (2005) e incluye todos los discursos de ese año de los miembros de la Cámara de Representantes y del Senado.

El texto ya está tokenizado en *bigrams* , después de eliminar las *stopwords* y aplicar un algoritmo de lematización (usando el *Porter stemmer*).

In [ ]:
data("congress109", package = "textir")

La matriz `congress109Counts` contiene el número de veces que cada una de las 1000 frases más comunes fue utilizada en el 109º Congreso por cada uno de los 529 miembros del Congreso (Cámara y Senado). 

Es decir, cada documento corresponde a la transcripción combinada de todos los discursos de un solo orador.

Podemos inspeccionar los conteos de algunas frases específicas para dos oradores:

In [ ]:
congress109Counts[c("Barack Obama","John Boehner"),995:999]

Tambien tenemos información sobre  el partido que pertenecen el estado, la camara, y la proporcion de votos que obtuvo Bush en la eleccion presindecial en el distrito/estado de cada representante

In [ ]:
congress109Ideology[c("Barack Obama","John Boehner"),1:5]

Vamos a tratar de predecir estos votos  a partir de como los congresistas hablan. La idea es que el lenguage apela a los votantes: 

In [ ]:
as.matrix(congress109Counts[c("Tom Price","William Jefferson"),c("death.tax","estate.tax")])

In [ ]:
repshare <- congress109Ideology$repshare
X1 <- as.matrix(congress109Counts)
words<-colnames(X1)

covariance<-apply(X1,2,function(x) cor(x,repshare))

                  

wordcloud(words = words,
          freq = covariance,
          min.freq = 0,
          scale = c(1.5, 0.1), 
          max.words=200, 
          random.order=FALSE, 
          colors=brewer.pal(8, "YlOrRd"))

### Regresión 

$$
RepShare= X\beta  + u
$$

In [ ]:
# Predictores
X <- as(congress109Counts, "dMatrix")

# Outcome
repshare <- congress109Ideology$repshare

In [ ]:
# lasso
lassoslant <- cv.gamlr(X>0, repshare)  

B <- coef(lassoslant$gamlr)[-1,] 

In [ ]:
tail(sort(round(B[B!=0],4)),10)

In [ ]:
head(sort(round(B[B!=0],4)),10)

## Modelado de topicos o temas 

### Ejemplo comentarios en we8there

Para estudiar la factorización de texto, pasaremos de la política a los restaurantes. Contamos con 6166 reseñas, con una extensión promedio de 90 palabras por reseña, del ya desaparecido sitio web de viajes `we8there.com`. 

In [ ]:
data("we8there", package = "textir")

Una característica útil de estas reseñas es que contienen texto y una calificación multidimensional sobre la experiencia general, el ambiente, la comida, el servicio y la relación calidad-precio. Cada aspecto se califica en una escala de cinco puntos, donde 1 indica pésimo y 5 indica excelente. 

Por ejemplo, un usuario envió una reseña muy positiva para Waffle House #1258 en Bossier City, Luisiana: 

Waffle House #1258 in Bossier City, Louisiana:

*I normally would not revue a Waffle House but this one deserves it. The workers, Amanda, Amy, Cherry, James and J.D. were the most pleasant crew I have seen. While it was only lunch, B.L.T. and chili, it was great. The best thing was the 50’s rock and roll music, not to loud not to soft. This is a rare exception to what you all think a Waffle House is. Keep up the good work.*

*Overall: 5, Atmosphere: 5, Food: 5, Service: 5, Value: 5.*



Otro usuario encontró que Sartin's Seafood, no es muy bueno:

Sartin’s Seafood in Nassau Bay, Texas,

*Had a very rude waitress and the manager wasn’t nice either.*
*Overall: 1, Atmosphere: 1, Food: 1, Service: 1, Value: 5.*

In [ ]:
x <- we8thereCounts
x[1,x[1,]!=0]

### PCA

In [ ]:
pca <- prcomp(x, scale=TRUE) # cuidado demora mucho

In [ ]:
tail(sort(pca$rotation[,1]))

In [ ]:
tail(sort(pca$rotation[,4]))

### LDA

#### Aside: Ejemplo paso a paso

En este ejemplo vamos a asumir que hay 3 (D), que contienen 3 temas (K) y 7 (V) palabras

- Paso 1: Definir el vocabulario

In [ ]:
set.seed(123)

vocabulario <- c("gato", "perro", "vacuna", "hospital", "dinero", "banco", "impuestos")
V <- length(vocabulario)
V

- Paso 2: Definir las distribuciones de palabras por tópico (φ)

Estas salen de una Dirichlet, mas sobre esto abajo

In [ ]:
# Cada vector debe tener longitud igual al vocabulario

phi_1 <- c(0.4, 0.4, 0.05, 0.05, 0.03, 0.03, 0.04)  # Animales
phi_2 <- c(0.05, 0.05, 0.4, 0.4, 0.03, 0.03, 0.04)  # Salud
phi_3 <- c(0.05, 0.05, 0.05, 0.05, 0.35, 0.35, 0.10)  # Economía

# Guardamos las distribuciones en una lista
phi <- list(phi_1, phi_2, phi_3)


phi

- Paso 3: Definir mezcla de tópicos del documento (θ)

Ahora si veamos como funciona la Dirichlet. Esta distribución es una de las pocas distribuciones que modela directamente vectores de probabilidades. Es decir, vectores cuyas componentes:

- son no negativas,

- y suman exactamente 1.

In [ ]:

p_load("ggtern")
p_load("MCMCpack")

# Número de muestras
set.seed(123)
D <- 3 #numero de documentos

# Parámetros de la Dirichlet
alpha <- c(1, 1, 20)  # K=3

# Generar muestras
samples <- rdirichlet(D, alpha)
df <- data.frame(Topic1 = samples[,1], Topic2 = samples[,2], Topic3 = samples[,3])

# Graficar puntos de masa
ggtern(data = df, aes(x = Topic1, y = Topic2, z = Topic3)) +
  geom_point(alpha = 0.5, color = "blue", size = 1.5) +
  theme_bw() +
  labs(title = "Muestras de una Dirichlet",
       x = "Tópico 1", y = "Tópico 2", z = "Tópico 3")


In [ ]:
df

In [ ]:
theta_d <- c(df[1,1], df[1,2], df[1,3])  # de arriba

theta_d

- Paso 4: Número de palabras en el documento

In [ ]:

N <- 10


- Paso 5: Generar los tópicos z_1, ..., z_N

In [ ]:


z <- sample(1:3, size = N, replace = TRUE, prob = theta_d)

z

- Paso 6: Generar palabras según los tópicos

In [ ]:

w <- character(N)
for (n in 1:N) {
  w[n] <- sample(vocabulario, size = 1, prob = phi[[z[n]]])
}

# Resultado: palabras generadas
print(w)

- Paso 7: Bag of Words (conteo por palabra)

In [ ]:


bow <- table(factor(w, levels = vocabulario))
print(bow)


#### Ingenieria reversa

Hasta ahora, simulamos documentos **generando** palabras. Pero en la realidad en la pratica

* Un corpus de documentos reales → textos observados.
* Una representación en BoW o DTM.


Lo que buscamos es inferir los **parámetros** del modelo:

  * Las mezclas de tópicos por documento $\theta_d$,
  * Las distribuciones de palabras por tópico $\phi_k$,
  * Las asignaciones de tópicos $z_{d,n}$ para cada palabra.

Es la ingenieria reversa de MLE: queremos encontrar los parámetros que hacen más **verosímil** (más probable) haber observado nuestros documentos.


$$
\max_{\theta, \phi} \ p(\text{documentos} \mid \theta, \phi)
$$

Pero en LDA no observamos los $z_{d,n}$, ni $\theta$, ni $\phi$. 

**Solo observamos las palabras.**

El problema a resolver es la siguiente verosimilitud:

$$
p(\mathbf{w} \mid \alpha, \beta) = \int \sum_{\mathbf{z}} p(\mathbf{w}, \mathbf{z}, \theta, \phi \mid \alpha, \beta) \ d\theta \ d\phi
$$

Este término **no se puede calcular exactamente**: la suma sobre todas las posibles asignaciones de tópicos $\mathbf{z}$ es **exponencial**.

¿Por qué es "exponencial"?

Porque si: hay $N$ palabras en total en el corpus,  y cada palabra puede tener **uno de $K$ tópicos**, entonces hay: $K^N$ posibles combinaciones de asignaciones de tópicos. Ejemplo: supongamos

* 2 documentos, cada uno con 10 palabras → $N = 20$,
* 3 posibles tópicos → $K = 3$

Entonces hay:

$$
3^{20} = 3,486,784,401
$$

combinaciones posibles de $\mathbf{z}$. 

Entonces

* No podemos **sumar sobre todas** esas posibilidades explícitamente.
* Por eso necesitamos métodos que **aproximen esa suma**:


En la practica hay varios métodos de **inferencia aproximada**, que estiman los parámetros.

Los dos más comunes:

1. **Gibbs Sampling** (una forma de MCMC)
2. **Variational Inference** (una versión determinista, más rápida)


#### Regreso al ejemplo de `we8there`

Vamos a usar `maptpx` de Matt Taddy. El algoritmo esta descipto ["On Estimation and Selection for Topic Models"](https://arxiv.org/pdf/1109.4518), e implementa "Topic Posterior Estimation" usando inferiencia variacional y nos aproxima una solución a:

$$
p(\mathbf{w} \mid \alpha, \beta) 
$$

In [ ]:
p_load("maptpx") # para modelar topicos

x <- as.simple_triplet_matrix(we8thereCounts) #Convierte a formato sparse
# Es una estructura sparse  que representa solo los elementos distintos de cero (non-sparse), usando tres vectores:
# i: los índices de fila
# j: los índices de columna
#v: los valores (conteos)

In [ ]:
tpc <- topics(x,K=10) 

Podemos comparar multiples K, para determinar el número óptimo de tópicos

In [ ]:
tpcs <- topics(x,K=5*(1:5), verb=1)

El Bayes Factor aqui se refiere a 



$$
 \exp\left(- \text{BIC}\right)  \approx p(\mathbf{w} \mid \alpha, \beta)
$$

entonces:

$$
\log p(\mathbf{w} \mid \alpha, \beta) \approx - \text{BIC}
$$

Entonces si nosotros buscabamos minimizar el BIC, vamos a quere maximizar el BF.

*Aside*: el BIC aqui va a ser

$$
\text{BIC} =  \log \hat{L} - \frac{1}{2} \left[ K(V - 1) + D(K - 1) \right] \cdot \log(n)
$$

* $\hat{L}$: log-likelihood del modelo (`mod$loglik` en `maptpx`)
* $V$: número de palabras en el vocabulario
* $D$: número de documentos
* $K$: número de tópicos
* $n$: número total de palabras en el corpus


La maximización del Bayes Factor en `maptpx` encuentra un valor de $K$ que funciona bien para una variedad de tareas posteriores, aunque a veces tiende a seleccionar un $K$ **más pequeño del que nos gustaría** para fines de *storytelling*

**Como se relaciona con otras medidas?**

##### Perplexity

La **perplejidad** (perplexity) es una transformación de la log-verosimilitud, usada como medida de **cuán bien el modelo predice nuevas palabras**.

Se define como:

$$
\text{Perplexity}(\mathbf{w}) = \exp\left( - \frac{1}{N} \log p(\mathbf{w}) \right)
$$

* Cuanto **más baja**, **mejor el modelo predice**.
* Se puede calcular sobre un conjunto de test → mide **poder predictivo**.

##### Coherence
* **Coherencia** mide **cuán interpretables y temáticamente consistentes** son los tópicos. Lo hace calculado qué tan frecuentemente **las palabras más probables de un tópico coocurren en los mismos documentos**. Si las palabras clave de un tópico tienden a aparecer juntas, decimos que el tópico es **coherente**.

* No depende de la log-verosimilitud ni del Bayes factor.
* Se calcula a partir de:

  * las palabras más frecuentes de cada tópico,
  * y su coocurrencia en los documentos.

* Mide si los tópicos "tienen sentido" según cómo las palabras aparecen juntas en los textos.

Formula: 
* Tópico $k$ tiene palabras $w_1, w_2, \dots, w_M$
* Cada $w_i$ es una palabra del vocabulario (por ejemplo, las n más probables según $\phi_k$)

Entonces:

$$
\text{Coherencia}(k) = \sum_{i < j} \log \left( \frac{D(w_i, w_j) + \epsilon}{D(w_j)} \right)
$$

donde:

* $D(w_j)$: número de documentos que contienen la palabra $w_j$
* $D(w_i, w_j)$: número de documentos que contienen ambas $w_i$ y $w_j$
* $\epsilon$: pequeño valor para evitar log(0), usualmente 1

*Ejemplo*: 
Supongamos que un tópico tiene como palabras más probables:

$$
\text{"hospital", "médico", "enfermera", "paciente"}
$$

Estas palabras suelen aparecer juntas en los documentos → el tópico es coherente.

Ahora imagina:

$$
\text{"hospital", "perro", "impuestos", "fútbol"}
$$

Estas palabras no aparecen juntas normalmente → el tópico es incoherente.

##### Interpretación

La interpretación de los tópicos se realiza de manera similar a como se hace con PCA

Podemos comenzar observando las "palabras principales" de cada tópico.

Pero para que esto sea útil, hay que tener cuidado con el criterio que se usa para ordenar las palabras consideradas "principales".

Si las ordenás por la **probabilidad de palabra en el tópico** ($\phi_{kj}$), terminarás con palabras que son frecuentes en el tópico $k$, pero que también pueden ser comunes en otros tópicos —esto pasa especialmente si eliminaste solo un pequeño conjunto de *stopwords*.

En su lugar, la función `summary()` de `maptpx` ordena las palabras según el **lift**:

$$
\text{lift}_{kj} = \frac{\phi_{kj}}{\bar{x}_j} = \frac{\text{especificidad en el tópico}}{\text{frecuencia global}}
$$

donde $\bar{x}_j$ es la frecuencia promedio de la palabra $j$ en el corpus (es decir, su proporción promedio entre los documentos).

Este **lift** será alto para palabras que son **mucho más frecuentes en el tópico $k$** de lo que son en el lenguaje general del corpus.
Por eso, usar lift ayuda a resaltar palabras **más exclusivas o distintivas del tópico**, y no simplemente comunes en todo el corpus.


* Si $\text{lift}_{kj} > 1$: la palabra es **más característica** del tópico que del corpus.
* Si $\text{lift}_{kj} \ll 1$: es una palabra común (no distintiva).

El lift evita que en los tópicos aparezcan como “principales” palabras genéricas, que aunque sean frecuentes en todos los documentos, **no ayudan a interpretar el tópico en cuestión**.


In [ ]:
summary(tpcs, n=10) 

El primer tópico contiene retroalimentación positiva, y por eso tiene la misma interpretación que el primer componente principal (PC1).

In [ ]:
rownames(tpcs$theta)[order(tpcs$theta[,1], decreasing=TRUE)[1:10]]

Pero los otros tópicos parecen distintos y más interpretables que los factores obtenidos mediante PCA.
Por ejemplo:

El tópico 2 trata sobre tener que esperar,

In [ ]:
rownames(tpcs$theta)[order(tpcs$theta[,2], decreasing=TRUE)[1:10]]

El tópico 3 incluye reseñas positivas de clientes frecuentes.

In [ ]:
rownames(tpcs$theta)[order(tpcs$theta[,3], decreasing=TRUE)[1:10]]

Podemos comparar los **scores de tópicos** (los $\theta$) con las **calificaciones de las reseñas**.

Veamos la **calificación general** en función de los **scores de los documentos** en el primer topico


In [ ]:
boxplot(tpcs$omega[,1] ~ we8thereRatings$Overall, col="gold", xlab="overall rating", ylab="topic 1 score")

Hay claramente una **relación positiva entre el tópico 1 y la calificación general**  que lo que se observaba usando el **primer componente principal (PC1)**. (HW)

y en el segundo vemos algo parecido:

In [ ]:
boxplot(tpcs$omega[,2] ~ we8thereRatings$Overall, col="pink", xlab="overall rating", ylab="topic 2 score")

Estas relaciones sugieren una estrategia de **regresión sobre tópicos** para predecir la **calificación de una reseña** a partir de su **contenido textual**.

#### Prediciendo los ratings en `we8there`

In [ ]:
stars <- we8thereRatings[,"Overall"]

In [ ]:
Xtopics<-as(tpcs$omega, "dMatrix")

#### Lasso temas

In [ ]:
p_load("gamlr")
regtopics.cv <- cv.gamlr(tpcs$omega, stars)

#### Lasso palabras 

In [ ]:
regwords.cv <- cv.gamlr(we8thereCounts, stars)

Resultados

In [ ]:
par(mfrow=c(1,2), mai=c(.3,.6,.7,.1), omi=c(.5,.2,0,0))
plot(regtopics.cv, ylim=c(1,2), xlab="", ylab="")
mtext("topic regression", font=2, line=2)
plot(regwords.cv, ylim=c(1,2), xlab="", ylab="")
mtext("token regression", font=2, line=2)
mtext(side=2, "mean squared error", outer=TRUE, line=0)
mtext(side=1, "log lamba", outer=TRUE, line=1)